# Clean the EIA Data By Merging Power Plant and Generator Data to Detail the Various Energy Sources Used by Each Plant

In [ ]:
import pandas as pd

In [ ]:
# Define a list of years to loop over
years = range(2014, 2024)

# List to store DataFrames for each year
merged_data = []

# Loop through each year
for year in years:
    # Read the plant and generator data for the current year
    plant_file = f"C:\\Users\\kaavy\\OneDrive\\Documents\\Sci4GA Internship\\DATA on electricity powerplants from EIA\\{year}\\2___Plant_Y{year}.xlsx"
    generator_file = f"C:\\Users\\kaavy\\OneDrive\\Documents\\Sci4GA Internship\\DATA on electricity powerplants from EIA\\{year}\\3_1_Generator_Y{year}.xlsx"
    
    # Read the data
    plant_og_df = pd.read_excel(plant_file)
    generator_df = pd.read_excel(generator_file)
    plant_df = plant_og_df[plant_og_df['State'] == 'GA']
    
    # Find unique values in the 'Technology' column
    unique_technologies = generator_df['Technology'].unique()
    unique_technologies = unique_technologies[~pd.isna(unique_technologies)]  # Remove NaN values

    # Group by 'Plant Code' and 'Technology', count generators
    tech_counts = generator_df.groupby(['Plant Code', 'Technology']).size().unstack(fill_value=0)

    # Merge aggregated technology data back to the plant_df
    merged_df = plant_df.merge(tech_counts, on='Plant Code', how='left')

    # Fill NaN values (if any) with 0 for generator counts
    merged_df.fillna(0, inplace=True)

    # Now select only the columns that exist in merged_df and correspond to the technologies
    valid_technologies = [tech for tech in unique_technologies if tech in merged_df.columns]

    # Use idxmax to find the dominant technology based on the valid columns
    # merged_df['Dominant Technology'] = merged_df[valid_technologies].idxmax(axis=1)

    # Add a 'Year' column
    merged_df['Year'] = year

    # Retain only relevant columns
    columns_to_keep = ['Plant Code', 'Plant Name', 'Latitude', 'Longitude', 'State', 'Year'] + valid_technologies
    merged_df = merged_df[columns_to_keep]

    # Append the merged data for the current year to the list
    merged_data.append(merged_df)

# Concatenate all the years into a single DataFrame
final_df = pd.concat(merged_data, ignore_index=True)
# Convert Year column to Date (set the first day of each year)
final_df['Year'] = pd.to_datetime(final_df['Year'], format='%Y')

# Save the concatenated DataFrame to an Excel file
#output_path = r"C:\Users\kaavy\OneDrive\Documents\Sci4GA Internship\Combined_Plant_Data_2014_to_2023.xlsx"
#final_df.to_excel(output_path, index=False)

#print(f"File with combined data saved at: {output_path}")

# Identify the Total Number of Generators for Each Energy Source from 2014 to 2023

In [ ]:
combined_plant = pd.read_excel(r"C:\Users\kaavy\OneDrive\Documents\Sci4GA Internship\Combined_Plant_Data_GA_2014_to_2023.xlsx")

In [ ]:
# Specify the columns to sum
target_columns = [
    'Conventional Steam Coal',
    'Conventional Hydroelectric',
    'Petroleum Liquids',
    'Natural Gas Fired Combined Cycle',
    'Other Waste Biomass',
    'Natural Gas Fired Combustion Turbine',
    'Natural Gas with Compressed Air Storage',
    'Natural Gas Internal Combustion Engine',
    'Nuclear',
    'Natural Gas Steam Turbine',
    'Onshore Wind Turbine',
    'All Other',
    'Landfill Gas',
    'Municipal Solid Waste',
    'Geothermal',
    'Hydroelectric Pumped Storage',
    'Batteries',
    'Solar Photovoltaic',
    'Wood/Wood Waste Biomass',
    'Other Gases',
    'Other Natural Gas',
    'Petroleum Coke',
    'Coal Integrated Gasification Combined Cycle',
    'Solar Thermal without Energy Storage',
    'Solar Thermal with Energy Storage',
    'Flywheels',
    'Offshore Wind Turbine'
]

# Group by year and sum the target columns
result = combined_plant.groupby('Year')[target_columns].sum()

# Reset index for a cleaner output
result = result.reset_index()

# Display the result
print(result)

# Collect Monthly Energy Generation Data for Georgia from EIA

In [ ]:
import requests # Library for making HTTP requests to the API
import pandas as pd

api_key = "TYbnbyZAB6i2W9v3hzyqHXTyAhQufNdjPdJYYA2Z"
endpoint = "https://api.eia.gov/v2/electricity/facility-fuel/data"

# Initialize empty list to store all data
all_data = [] # Empty list to store all retrieved data
offset = 0 # Starting point for pagination, helps retrieve data in chunks of 5000 records

while True:
    params = {
        "api_key": api_key,
        "frequency": "monthly", # Data granularity
        "data[0]": "generation",
        "data[1]": "gross-generation",
        "facets[state][]": "GA",
        "start": "2014-01",
        "end": "2023-12",
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": offset,
        "length": 5000 # Maximum records per request
    }

    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        data = response.json()["response"]["data"]
        
        if not data:  # If no more data is returned, break the loop
            break
            
        all_data.extend(data)
        offset += 5000
        print(f"Retrieved {len(all_data)} records...")
        
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        break

# Create final dataframe
df = pd.DataFrame(all_data)
df['period'] = pd.to_datetime(df['period'])
df = df.sort_values('period', ascending=True)
df.to_csv('georgia_generation_complete.csv', index=False)

print("\nData retrieval complete!")
print(f"Total records retrieved: {len(df)}")
print("\nDate range in data:")
print(f"From: {df['period'].min()}")
print(f"To: {df['period'].max()}")

In [ ]:
generation_df=pd.read_csv(r"C:\Users\kaavy\OneDrive\Documents\Sci4GA Internship\georgia_generation_complete.csv")

## Add location information to Energy Generation Data

In [ ]:
# Ensure plant_df 'Year' is converted to a consistent format (integer)
plant_df['Year'] = pd.to_datetime(plant_df['Year']).dt.year

# Extract the year from the 'period' column in generation_df
generation_df['Year'] = pd.to_datetime(generation_df['period']).dt.year

# Merge on both 'plantCode' and 'Year'
merged_df = pd.merge(
    generation_df, 
    plant_df[['Plant Code', 'Year', 'Latitude', 'Longitude']], 
    left_on=['plantCode', 'Year'], 
    right_on=['Plant Code', 'Year'], 
    how='left'
)

# Drop redundant 'Plant Code' column (if necessary)
merged_df.drop(columns='Plant Code', inplace=True)

# Save the result to a CSV file
# merged_df.to_csv('GA_Generation_with_Location_By_Year.csv', index=False)

# Examine How Much Emissions Each Energy Source Releases When Producing Energy

In [ ]:
emissions_df = pd.read_excel(r"C:\Users\kaavy\OneDrive\Documents\Sci4GA Internship\EGRID 2022 Data.xlsx")
emissions_ga_df = emissions_df[emissions_df['Plant state abbreviation'] == 'GA']

In [ ]:
emissions_summary = emissions_ga_df.groupby('Plant primary fuel category')[[
    'Plant annual NOx emissions (tons)', 
    'Plant annual SO2 emissions (tons)', 
    'Plant annual CO2 emissions (tons)'
]].sum().reset_index()

# Optional: Sort by CO2 emissions (or any other column) in descending order
emissions_summary = emissions_summary.sort_values(by='Plant annual CO2 emissions (tons)', ascending=False)

# Display the result
print(emissions_summary)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Input the data
data = {
    "Energy Source": ["Coal", "Solar", "Natural Gas", "Biomass", "Hydroelectric", "Petroleum", "Nuclear"],
    "Gross Generation (MWh)": [361219.26, 141249.96, 1208943.9, 1169414.38, 984373.88, 111704.89, 683980],
    "Annual NOx Emissions (tons)": [12228.074, 0.0, 3977.661, 4073.215, 0.0, 770.054, 0.0],
    "Annual SO2 Emissions (tons)": [6921.862, 0.0, 219.629, 3990.100, 0.0, 580.035, 0.0],
    "Annual CO2 Emissions (tons)": [19299720.0, 0.0, 26550080.0, 518810.7, 0.0, 252900.5, 0.0],
    "Count of Plants": [2, 114, 43, 16, 30, 75, 2]
}


# Create the DataFrame
df = pd.DataFrame(data)

# Calculate Emissions Intensity (Emissions per MWh)
df["NOx Intensity (tons/MWh)"] = df["Annual NOx Emissions (tons)"] / df["Gross Generation (MWh)"]
df["SO2 Intensity (tons/MWh)"] = df["Annual SO2 Emissions (tons)"] / df["Gross Generation (MWh)"]
df["CO2 Intensity (tons/MWh)"] = df["Annual CO2 Emissions (tons)"] / df["Gross Generation (MWh)"]



### 2. Heatmap for NOx and SO2 Intensity
# Melt the DataFrame to reshape for the heatmap
nox_so2_df = df[["Energy Source", "NOx Intensity (tons/MWh)", "SO2 Intensity (tons/MWh)"]]
nox_so2_df_melted = nox_so2_df.melt(id_vars=["Energy Source"], 
                                    var_name="Emission Type", 
                                    value_name="Intensity")

plt.figure(figsize=(12, 6))
sns.heatmap(
    nox_so2_df_melted.pivot("Energy Source", "Emission Type", "Intensity"),
    annot=True, fmt=".2e", cmap="Blues", cbar_kws={'label': 'Intensity (tons/MWh)'}
)
plt.title("NOx and SO2 Intensity Heatmap by Energy Source")
plt.xlabel("Emission Type")
plt.ylabel("Energy Source")
plt.show()

In [ ]:
# Calculate Emissions Intensity (Emissions per MWh)
df["CO2 Intensity (tons/MWh)"] = df["Annual CO2 Emissions (tons)"] / df["Gross Generation (MWh)"]

### 1. Heatmap for CO2 Intensity with one Emission type on X-axis
# Adding a placeholder column to make it similar to the NOx and SO2 layout
df_co2 = df[["Energy Source", "CO2 Intensity (tons/MWh)"]]
df_co2["Emission Type"] = "CO2 Intensity"

# Plot Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(
    df_co2.pivot("Energy Source", "Emission Type", "CO2 Intensity (tons/MWh)"),
    annot=True, fmt=".2e", cmap="Blues", cbar_kws={'label': 'CO2 Intensity (tons/MWh)'}
)
plt.title("CO2 Intensity Heatmap by Energy Source")
plt.xlabel("Emission Type")
plt.ylabel("Energy Source")
plt.show()